In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

def get_df():
    sale_path = "/home/akshat/Documents/projects/joshi-uncle/data/REPORT_2222.XLS"
    df = pd.read_excel(sale_path, sheet_name="MARG ERP 9+ Excel Report", header=None)
    return df

In [2]:
df = get_df()
print(df.shape)
df.head()

(731, 20)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,IKSHANA MEDICAL AND SURGICAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"1ST FLO KOHLI TOWERS THANDI ROAD CIVIL LINES,H...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Phone : 9058591180 E-Mail : IKSHANAMEDICALANDS...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FORM 29 01/04/2023 - 17/02/2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DATE,BILL NO.,DATED PARTY NAME,GST.No.,BILL VALUE,[------CP-GST 5,%------],[-----CP-GST 12,%-----],[-----CP-GST,18,%-----][------------LP-GST,5,%-,-----------],[-----------LP-GST 12,NaN,%-----------],[-----------LP-GST 18,%-----------]


In [3]:
df1 = df.iloc[6:, :]
print(df1.shape)
df1.head()

(725, 20)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
6,01/04/2023,61285,01/04 DENIS PHARMACEUTICALS,05AAGFD1879E1ZK,7077,0,0,0,0,0,0,0,0,0,6318.92,379.14,379.14,0,0,0
7,01/04/2023,GST-23-1,01/04 OM SAI VISION,09AAFFO6811L1ZX,2483,0,0,2217,266.04,0,0,0,0,0,0,0,0,0,0,0
8,01/04/2023,GST-23-10,01/04 OM SAI VISION,09AAFFO6811L1ZX,2483,0,0,2217,266.04,0,0,0,0,0,0,0,0,0,0,0
9,01/04/2023,GST-23-11,01/04 OM SAI VISION,09AAFFO6811L1ZX,3240,3085.7,154.28,0,0,0,0,0,0,0,0,0,0,0,0,0
10,01/04/2023,GST-23-2,01/04 OM SAI VISION,09AAFFO6811L1ZX,2483,0,0,2217,266.04,0,0,0,0,0,0,0,0,0,0,0


In [20]:
# Initialize variables to keep track of current bill date and bill number
current_bill_date = None
current_bill_no = None

columns=[
    "Inv No.",
    "Inv Date",
    "Party/Cash",
    "Product's Name",
    "HSN Code",
    "Qty",
    "Rate",
    "GST%",
    "Amount",
    "CGST %",
    "CGST Amt",
    "SGST %",
    "SGST Amt",
    "IGST Amt",
    "Discount %",
    "Total",
    "State",
    "Party's GST",
    "Reg Type",
    "Place of Supply",
    "Country",
    "Consignee State",
    "Consignee GST",
]


result_df = pd.DataFrame(
    columns= columns
)



def get_transformed_rows(row):
    # Extract required values from the row
    inv_date = row.iloc[0]
    inv_no = row.iloc[1]
    party_or_cash = row.iloc[2]
    gst_no = row.iloc[3]

    # if start with 05 then calculate cgst/sgst
    # if not start with 05 then calculate igst
    calculate_igst = not (str(gst_no).strip().startswith("05"))

    amount_5 = float(row.iloc[11])
    amount_12 = float(row.loc[14])
    amount_18 = float(row.loc[17])

    new_rows = []
    for per, amount in [(5, amount_5), (12, amount_12), (18, amount_18)]:
        cgst_per = (per / 2) if (not calculate_igst) else 0.0
        sgst_per = (per / 2) if (not calculate_igst) else 0.0

        cgst_amt = amount * (cgst_per / 100) if not calculate_igst else 0.0
        sgst_amt = amount * (sgst_per / 100) if not calculate_igst else 0.0
        igst_amt = (amount * per / 100) if calculate_igst else 0.0
        discount = 0.0

        new_row = {
            "Inv No.": inv_no,
            "Inv Date": inv_date,
            "Party/Cash": party_or_cash,
            "Product's Name": "",
            "HSN Code": "",
            "Qty": "",
            "Rate": "",
            "GST%": per,
            "Amount": amount,
            "CGST %": cgst_per,
            "CGST Amt": cgst_amt,
            "SGST %": sgst_per,
            "SGST Amt": sgst_amt,
            "IGST Amt": igst_amt,
            "Discount %": discount,
            "Total": amount + cgst_amt + sgst_amt + igst_amt - discount,
            "State": "Uttarakhand",
            "Party's GST": gst_no,
            "Reg Type": "",
            "Place of Supply": "Uttarakhand",
            "Country": "India",
            "Consignee State": "Uttarakhand",
            "Consignee GST": "",
        }
        new_rows.append(new_row)

    return new_rows


# Iterate over rows
for index, row in df1.iterrows():
    # print(str(row.iloc[0]), str(row.iloc[0]).startswith("A"))
    # If the row starts with a date (assuming the date is in the first cell)
    if not pd.notna(row.iloc[0]): continue
    
    # Check for date in 1st column
    try:
        pd.to_datetime(row.iloc[0], format="%d/%m/%Y").strftime(format= "%d/%m/%Y")
    except Exception as e:
        continue

    # If the row starts with a bill number
    extracted_rows = get_transformed_rows(row)
    result_df = pd.concat(
        [
            result_df,
            pd.DataFrame(extracted_rows),
        ],
        ignore_index=True,
    )

In [5]:
result_df

,Inv No.,Inv Date,Party/Cash,Product's Name,HSN Code,Qty,Rate,GST%,Amount,CGST %,CGST Amt,SGST %,SGST Amt,IGST Amt,Discount %,Total,State,Party's GST,Reg Type,Place of Supply,Country,Consignee State,Consignee GST
0,61285,01/04/2023,01/04 DENIS PHARMACEUTICALS,,,,,5,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,Uttarakhand,05AAGFD1879E1ZK,,Uttarakhand,India,Uttarakhand,
1,61285,01/04/2023,01/04 DENIS PHARMACEUTICALS,,,,,12,6318.92,0.0,0.0,0.0,0.0,758.2704,0.0,7077.1904,Uttarakhand,05AAGFD1879E1ZK,,Uttarakhand,India,Uttarakhand,
2,61285,01/04/2023,01/04 DENIS PHARMACEUTICALS,,,,,18,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,Uttarakhand,05AAGFD1879E1ZK,,Uttarakhand,India,Uttarakhand,
3,GST-23-1,01/04/2023,01/04 OM SAI VISION,,,,,5,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,Uttarakhand,09AAFFO6811L1ZX,,Uttarakhand,India,Uttarakhand,
4,GST-23-1,01/04/2023,01/04 OM SAI VISION,,,,,12,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,Uttarakhand,09AAFFO6811L1ZX,,Uttarakhand,India,Uttarakhand,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1708,GST-2389,17/02/2024,17/02 SHREE SAI TRADING CO.,,,,,12,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,Uttarakhand,09BBWPS9350K1ZI,,Uttarakhand,India,Uttarakhand,
1709,GST-2389,17/02/2024,17/02 SHREE SAI TRADING CO.,,,,,18,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,Uttarakhand,09BBWPS9350K1ZI,,Uttarakhand,India,Uttarakhand,
1710,LKO-23-096,17/02/2024,17/02 APPASAMY ASSOCIATES,,,,,5,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,Uttarakhand,09AASCA3660R1ZR,,Uttarakhand,India,Uttarakhand,
1711,LKO-23-096,17/02/2024,17/02 APPASAMY ASSOCIATES,,,,,12,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,Uttarakhand,09AASCA3660R1ZR,,Uttarakhand,India,Uttarakhand,


In [22]:
result_df.to_excel('purchase_transformed.xlsx', index=False)


In [44]:
# total_row = {c : "" for c in  columns}
# total_row['Amount'] = result_df['Amount'].sum()
# total_row['CGST Amt'] = result_df['CGST Amt'].sum()
# total_row['SGST Amt'] = result_df['SGST Amt'].sum()
# total_row['IGST Amt'] = result_df['IGST Amt'].sum()
# total_row['Total'] = result_df['Total'].sum()
# total_row['Party/Cash'] = "Total"


# result_df = pd.concat(
#     [
#         result_df,
#         pd.DataFrame([total_row]),
#     ],
#     ignore_index=True,
# )

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('pandas_simple.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
result_df.to_excel(writer, sheet_name='Sheet1')

# Get the xlsxwriter objects from the dataframe writer object.
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

In [45]:
worksheet.dim_colmax, worksheet.dim_rowmax

(23, 165)

In [47]:
from openpyxl.utils import get_column_letter

amt_index = result_df.columns.get_loc("Amount")

amt_col_letter = get_column_letter(amt_index)

In [49]:

get_column_letter(100)
worksheet.write_formula(f'{amt_col_letter}{worksheet.dim_rowmax + 1}', f'=SUM({amt_col_letter}0:{amt_col_letter}{worksheet.dim_rowmax})')

0

In [51]:
workbook.save

AttributeError: 'Workbook' object has no attribute 'save'